# The Weil Pairing

**Module 07** | 07-pairings

*Divisors intro, weil_pairing()*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **the weil pairing**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Bilinear Maps](07a-bilinear-maps-definition.ipynb).
- Concepts and notation introduced in the previous notebook.

## Divisors on Curves

In [ ]:
# A divisor is a formal sum of points on the curve
# TODO: introduce divisors with simple examples

## Computing the Weil Pairing

In [ ]:
# SageMath's weil_pairing() for torsion points
# TODO: compute Weil pairing on a small curve

## Properties of the Weil Pairing

In [ ]:
# Alternating: e(P, P) = 1
# TODO: verify key properties of the Weil pairing

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **the weil pairing**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [Pairing-Friendly Curves](07c-pairing-friendly-curves.ipynb)